# CSS Lab: Social Rank and Hierarchy
Network analysis can be used to examine social status within a community. This lab will use data from the 1995-1997 Teenage Friends and Lifestyle study [MA1997]. This study collected friendship and other data from a group of teenage students over the course of three years.

## Contents
1. Setup
2. Directed networks
    1. Load and visualize the social network
3. Social status
    1. Minimum violation rankings
    2. Eigenvector centrality
    3. Spring rank
    4. Compare MVR approximations
    5. Analyze friendships

## Setup

In [ ]:
%pylab inline
import itertools
import json
import math
import sys
import time
import urllib.request
import networkx as nx
import networkx.algorithms as nxalg
import networkx.algorithms.community as nxcom
import networkx.readwrite as nxrw
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as spstats
import visJS2jupyter.visJS_module as vjs
import re
from springrank.SpringRank_tools import SpringRank
from springrank.tools import build_graph_from_adjacency


In [ ]:
# Helper functions

def load_tfls(wave=1):
    G = nx.DiGraph()
    with open("external/TFLS/friendship-{}.csv".format(wave)) as f:
        # Read header
        labels = re.split(",", f.readline().strip())[1:]
        labels = [x.strip('"') for x in labels]
        # Read data
        for row, row_data in enumerate(f):
            cells = re.split(",", row_data.strip())
            row_label = cells[0].strip('"')
            cells = cells[1:]
            for col, cell_data in enumerate(cells):
                if row_label == labels[col]:
                    continue
                if cell_data == "1" or cell_data == "2":
                    cell_data = 1
                    G.add_edge(row_label, labels[col], weight=cell_data, value=cell_data)
    nx.set_node_attributes(G, dict((v, v) for v in G.nodes()), name="label")
    return G

def get_colors():
    phi = (1 + math.sqrt(5)) / 2
    color = []
    for i in range(1, 20):
        theta = phi * i * math.pi * 2
        x = 128 + math.floor(64*math.sin(theta))
        y = 128 + math.floor(64*math.cos(theta))
        color.append((x, x, y))
    return color

def visualize_visjs(
        G, communities=None, colors=None, default_color="192,192,192",
        node_size_field="node_size", layout="spring", scale=500, pos=None,
        groups=None, smooth=False, weight=None, labels=dict(), title="",
        shadow=True, shape="dot", node_colors=dict(), node_alpha=1.0,
        edge_colors=dict()):
    # Get list of nodes and edges
    nodes = list(G.nodes())
    edges = list(G.edges())
    # Change node shapes for bipartite graph
    if groups is None:
        node_shapes = dict()
        node_sizes = dict()
    else:
        node_shapes = dict((n, "square") for n in groups)
        node_sizes = dict((n, 15) for n in groups)
        node_colors = dict((n, "192,128,0") for n in groups)
    # Per-node properties
    nodes_dict = dict((n, {
        "id": str(labels.get(n, n)),
        "node_size": node_sizes.get(n, 5),
        "node_shape": node_shapes.get(n, shape),
        "border_width": 2,
        "color": "rgba({},{})".format(node_colors.get(n, default_color), node_alpha)
        }) for n in nodes)
    if shape == "cat":
        for key, node in nodes_dict.items():
            node["node_shape"] = "image"
            node["node_image"] = "cat.png"
    # Generate a layout for the nodes
    edge_smooth_enabled = smooth
    edge_width = 4
    edge_arrow_scale = 2
    if communities is not None and pos is None:
        # Generate initial positions based on community
        phi = 3.14 / len(nodes)
        community_node = []
        # Create list of nodes and their communities
        for i, com in enumerate(sorted(communities, key=lambda x: len(x), reverse=True)):
            for node in com:
                community_node.append((i, node))
        # Sort by community and
        community_node = sorted(community_node)
        # Generate initial position by placing communities around a circle
        pos = dict((d[1], (math.cos(i*phi), math.sin(i*phi))) for i, d in enumerate(community_node))
    if layout == "circle":
        pos = nx.circular_layout(G, scale=scale)
    elif layout == "spring":
        pos = nx.spring_layout(G, k=3/math.sqrt(len(nodes)), scale=scale, pos=pos)
    # Assign position
    for n in nodes:
        nodes_dict[n]["x"] = pos[n][0]
        nodes_dict[n]["y"] = pos[n][1]
    # Calculate bounds for scaling
    x_min = min(pos.values(), key=lambda x: x[0])[0]
    x_max = max(pos.values(), key=lambda x: x[0])[0]
    y_min = min(pos.values(), key=lambda x: x[1])[1]
    y_max = max(pos.values(), key=lambda x: x[1])[1]
    x_range = x_max - x_min
    y_range = y_max - y_min
    max_range = max(x_range, y_range)
    # If we have communities, assign color based on community
    if colors is None:
        colors = ["{},{},{}".format(*c) for c in get_colors()]
    if communities is not None:
        for i, com in enumerate(sorted(communities, key=lambda x: len(x), reverse=True)):
            for node in com:
                try:
                    nodes_dict[node]["color"] = "rgba({},{})".format(colors[i], node_alpha)
                    nodes_dict[node]["color_index"] = i
                except IndexError:
                    nodes_dict[node]["color"] = "rgba({},{})".format(default_color, node_alpha)
    # Update color for bipartite nodes
    for node, node_attr in nodes_dict.items():
        if node in node_colors:
            node_attr["color"] = "rgba({},{})".format(node_colors.get(node, default_color), node_alpha)
    # Map node labels to contiguous ids
    node_map = dict(zip(nodes,range(len(nodes))))
    # Determine edge colors
    edge_colors_idx = {}
    for source, target in edges:
        source_color = nodes_dict[source].get("color_index", None)
        target_color = nodes_dict[target].get("color_index", None)
        if source_color == target_color and source_color is not None:
            edge_colors_idx[(source, target)] = source_color
    for e, c in edge_colors_idx.items():
        if c < len(colors) and e not in edge_colors:
            edge_colors[e] = colors[c]
    # Per-edge properties, use contiguous ids to identify nodes
    edge_scale = math.ceil(max_range / 200)
    edges_dict = []
    for source, target, data in G.edges(data=True):
        edge = {
            "source": node_map[source],
            "target": node_map[target],
            "title":'test',
            "color": "rgba({},0.3)".format(edge_colors.get((source,target), default_color)),
            "edge_width_field": "value",
            "value": data.get("value", 1) * edge_scale
        }
        edges_dict.append(edge)
    # Convert nodes dict to node list
    nodes_list = [nodes_dict[n] for n in nodes]
    # Check for directed graph
    if G.__class__ == nx.classes.digraph.DiGraph:
        directed = True
    else:
        directed = False
    # Call visjs
    return vjs.visjs_network(
        nodes_list, edges_dict,
        node_size_field="node_size",
        node_size_multiplier=10.0,
        node_shadow_enabled=shadow,
        node_color_border="rgba(0,0,0,{})".format(node_alpha),
        edge_width_field="value",
        edge_width=edge_width,
        edge_arrow_to=directed,
        edge_arrow_to_scale_factor=edge_arrow_scale,
        edge_smooth_enabled=edge_smooth_enabled,
        edge_smooth_type="curvedCW",
        graph_id=hash(title))

In [ ]:
visualize_visjs(nx.karate_club_graph(), title="karate", node_colors={0: "255,255,0"})

## Social status
When members of a social group can be ordered in terms of social status, it forms a pecking order. Directed network data can be used to uncover an underlying pecking order if it exists. An unreciprocated friendship can be a sign of a difference in social standing. The person who doesn't list the friendship may do so because they have too many friends to list, or because they don't want to list someone unpopular.

### Minimum violation rankings
In a perfect pecking order, friendships would only go from lower-ranked nodes to higher-ranked nodes.
So to find a pecking order, we look for a _minimum-violation ranking_: an ordering with as few links going from high-status to low-status individuals as possible.
Such an ordering may or may not exist.
The more violoations in the MVR, the less heirarchical a group is.
The example below allows you to test different orderings for the number of violations.

In [ ]:
# Helper functions 

def scale(x):
    return (x - np.mean(x)) / np.std(x)

def unit(rank):
    v = rank.values()
    span = max(v) - min(v)
    return dict((k, (x - min(v)) / span) for k, x in rank.items())

def giant_component(G):
    giant_component = sorted(nxalg.weakly_connected_components(G), reverse=True, key=len)[0]
    for v in set(G.nodes()) - giant_component:
        G.remove_node(v)
    return G

def get_mvr_rank(G, time_limit=60):
    # Iterative implementation to test all configurations
    labels = list(G.nodes())
    N = len(labels)
    ranking = []
    # Lists of remaining labels once i places have been assigned
    remaining = [[] for i in range(N)]
    remaining[0] = list(labels)
    # Upper limit is all pairs
    best_violations = N * (N - 1) / 2
    best_ranking = []
    start = time.time()
    last = start
    i = 0
    tried = 0
    while True:
        i += 1
        # Check for time limit, and sleep periodically to avoid locking cpu
        if i % 1000 == 0:
            t = time.time()
            delta = t - start
            if delta >= time_limit:
                print("Time limit exceeded")
                print("{} of {:0.2e} configurations in {:0.2f}s".format(i, math.factorial(N), delta))
                raise RuntimeError
            time.sleep(0)
        try:
            # Assign the next remaining label to the current place and move to the next
            ranking.append(remaining[len(ranking)].pop())
            if len(ranking) == N:
                # Out of places, compare the full assignment to the best so far
                trial_ranking = dict((label, i) for i, label in enumerate(ranking))
                rank_differences, violations = count_violations(G, trial_ranking)
                if violations < best_violations:
                    best_violations = violations
                    best_ranking = trial_ranking
                tried += 1
                # Backtrack
                ranking.pop()
            else:
                # Move to next place
                remaining[len(ranking)] = list(set(labels) - set(ranking))
        except IndexError:
            # No labels remaining, backtrack
            try:
                ranking.pop()
            except IndexError:
                # Unable to backtrack, all labels have been tried for all places
                break
    best_order = dict((x, N - i) for x, i in best_ranking.items())
    return best_ranking, best_order

def get_spring_rank(G):
    nodes = list(G.nodes())
    A=nx.to_numpy_matrix(G,nodelist=list(nodes))
    # Reverse spring rank so positive numbers represent higher status
    rank = dict(zip(nodes, -1 * SpringRank(A,alpha=0.0,l0=1.0,l1=1.0)))
    order = dict((elt[0], i) for i, elt in enumerate(sorted(rank.items(), key=lambda x: x[1])))
    return rank, order

def get_eigenvector_rank(G):
    all_nodes = set(G.nodes())
    components = nx.algorithms.components.weakly_connected_components(G)
    rank = dict()
    for c in components:
        H = G.copy()
        H.remove_nodes_from(all_nodes - set(c))
        component_rank = nx.algorithms.centrality.eigenvector_centrality_numpy(H)
        rank.update(component_rank)
    nodes, ranks = zip(*list(rank.items()))
    rank = dict(zip(list(nodes), list(ranks)))
    order = dict((elt[0], i) for i, elt in enumerate(sorted(rank.items(), key=lambda x: x[1])))
    return rank, order

def get_pagerank(G):
    rank = nx.pagerank(G)
    order = dict((elt[0], i) for i, elt in enumerate(sorted(rank.items(), key=lambda x: x[1])))
    return rank, order

def count_violations(G, order):
    rank_differences = []
    violations = 0
    for v, w in G.edges():
        difference = order[w] - order[v]
        rank_differences.append(difference)
        if difference < 0:
            violations += 1
    return rank_differences, violations
    
def plot_ordering(G, rank, title="Ordering", scale=150, shape="dot"):
    nodes = list(G.nodes())
    pos = dict((v, (0, r*scale)) for v, r in unit(rank).items())
    return visualize_visjs(G, layout=None, smooth=True, pos=pos, scale=scale, title=title, node_alpha=0.5, shape=shape)

def plot_orderings(Gs, ranks, title="Ordering", connect=False, scale=150, shape="dot", node_alpha=0.5):
    pos = dict()
    H = nx.DiGraph()
    all_nodes = set()
    for G in Gs:
        all_nodes |= set(G.nodes())
    for i, rank in enumerate(ranks):
        rank = unit(rank)
        GG = Gs[i]
        GG.add_nodes_from(all_nodes)
        new_labels = dict((v, "{}-{}".format(v, i)) for v in all_nodes)
        GG = nx.relabel.relabel_nodes(GG, new_labels)
        # Reverse y axis so higher rank is up
        pos_i = dict((new_labels[v], (i*scale, -1*r*scale)) for v, r in rank.items())
        pos.update(pos_i)
        H = nx.compose(H, GG)
    if connect:
        for i in range(len(ranks) - 1):
            for v in nodes:
                H.add_edge("{}-{}".format(v, i), "{}-{}".format(v, i+1))
    return visualize_visjs(H, layout=None, smooth=True, pos=pos, scale=scale, title=title, shape=shape, node_alpha=node_alpha, shadow=False)

def mvr_example():
    H = nx.DiGraph()
    H.add_edges_from([
        ('Lemon', 'Cotton'),
        ('Mia', 'Cotton'),
        ('Mia', 'Lemon'),
        ('Mia', 'Bagel'),
        ('Lemon', 'Bagel'),
        ('Setzer', 'Cotton'),
        ('Lemon', 'Setzer')
    ])
    return H

First, visualize the network.

In [ ]:
H = mvr_example()
visualize_visjs(H, shape="cat", scale = 250)

Next, let's define an ordering, visualize it, and count the number of violations.

In [ ]:
# To re-order the nodes, change these numbers
mvr_order = {
    "Bagel":  0,
    "Cotton": 1,
    "Lemon":  2,
    "Mia":    3,
    "Setzer": 4
}
# Count the violations and visualize
rank_differences, violations = count_violations(H, mvr_order)
print("Violations:", violations)
plot_ordering(H, mvr_order, scale=500, shape="cat", title="MVR Example")

Which edges count as violations? What is the lowest number of violations you can find? It might be helpful to move the nodes in the visualization. Try re-running the above cell after changing the rank numbers for each node. When you're done, run the cell below to compare your results to the minimum violation ranking.

In [ ]:
mvr_score, mvr_order = get_mvr_rank(H)
for label, rank in sorted(mvr_order.items(), key=lambda x: x[1]):
    print("Place {}: {}".format(rank, label))

The network above has a ranking with no violations. However, in real networks, it is uncommon to have a ranking with no violations, which is why the _minimum_ violation ranking is used.

## Teenage Friendship and Lifestyle Study
### Load and visualize the social network
Now we'll load and visualize friendship data from a real social network. The [MA1997] study determined friendships by asking participants to name their top friends. This method has in interesting feature: it is possible for a participant to list someone as a friend who does not list the participant as a friend. The friendship ties are _directed_. In the visualization, arrows go from the participant to the individuals they named as friends. Reciprocated friendships have arrows on both ends.

Study participants were asked to name up to 12 friends. Friends who were not study participants were not recorded. What are the benefits and drawbacks of this method of eliciting social networks? Can you think of another way to determine the friendship network of a group of people?

In [ ]:
G = load_tfls()
visualize_visjs(G, scale=1000, title="Full Network")

### Minimum Violation Ranking for TFLS
Now let's try to find the minimum violation ranking for the Teen Friendship and Lifestyle Study. The data set is much larger than the previous example, so it may take longer to run. If it takes longer than one minute to find an MVR, the function will exit with an error message. If the time limit is reached, use the data in the error message to calculate how long it would have taken.

In [ ]:
mvr_score, mvr_order = get_mvr_rank(G)

### Approximate MVRs

We'd like to find the MVR. Unfortunately, as we've seen above, finding the MVR is very difficult even with a lot of computing power (in computer science terms, it's NP-hard).
Several apporoximate methods exist.

The example below explores some possible approximations. Eigenvector centrality gives nodes a score based on how well-connected they are to other highly-connected nodes. PageRank (used by Google's search engine) is similar to eigenvector centrality, but is designed to handle disconnected networks. SpringRank [BLN2013] is specifically designed to find approximate MVRs. Each assigns a score to nodes, and then nodes are ranked according to that value. The visualization below shows (from left to right) eigenvector centrality, PageRank, and SpringRank. Higher nodes have higher scores, and arrows pointing downward represent violations.

In [ ]:
# Calculate the rank
er_score, er_order = get_eigenvector_rank(G)
sr_score, sr_order = get_spring_rank(G)
pr_score, pr_order = get_pagerank(G)
# Plot the orderings
plot_orderings([G, G, G], [er_score, pr_score, sr_score], title="MVR Examples", scale=2000, node_alpha=0.5)

### Compare approximations

The cells below count violations for rankings based on eigenvector centrality and SpringRank. Which works better? SpringRank is specifically designed to approximate MVRs, while eigenvector is not.

#### Eigenvector Centrality

In [ ]:
er_differences, er_violations = count_violations(G, er_order)
print("Violations:", er_violations)
print("Violation percent:", er_violations / len(list(G.edges())))
print("Mean rank difference:", np.mean(er_differences))

#### PageRank

In [ ]:
pr_differences, pr_violations = count_violations(G, pr_order)
print("Violations:", pr_violations)
print("Violation percent:", pr_violations / len(list(G.edges())))
print("Mean rank difference:", np.mean(pr_differences))

#### SpringRank

In [ ]:
sr_differences, sr_violations = count_violations(G, sr_order)
print("Violations:", sr_violations)
print("Violation percent:", sr_violations / len(list(G.edges())))
print("Mean rank difference:", np.mean(sr_differences))

Now let's see how highly correlated different scoring methods are. Each row/column below represents one scoring method. The off-diagonal squares plot different methods against each other, while the diagonals show histograms for each method.

In [ ]:
df = pd.DataFrame({
    "Eigenvector": er_score,
    "PageRank": pr_score,
    "SpringRank": sr_score
})
df["Eigenvector"] = np.log10(df["Eigenvector"])
df["PageRank"] = np.log10(df["PageRank"])
pd.plotting.scatter_matrix(df, figsize=(12,12))
plt.tight_layout()

In the cell below, select one of the scoring methods to use for the rest of the lab.

In [ ]:
#method = "Eigenvector"
method = "PageRank"
#method = "SpringRank"

if method == "Eigenvector":
    rank_differences, violations, get_rank = er_differences, er_violations, get_eigenvector_rank
elif method == "PageRank":
    rank_differences, violations, get_rank = pr_differences, pr_violations, get_pagerank
elif method == "SpringRank":
    rank_differences, violations, get_rank = sr_differences, sr_violations, get_spring_rank

### Analyze friendships

In [ ]:
bins = np.arange(min(rank_differences) - 0.5, max(rank_differences) + 0.5)
plt.hist(rank_differences, bins=bins)
xlabel("Rank difference"); ylabel("Count");
plt.tight_layout()

### Time evolution

In [ ]:
# Helper functions
def gradient(x):
    y = math.floor(128 - x*64)
    b = math.floor(128 + x*64)
    return "{},{},{}".format(y, y, b)

def plot_ordering_evolution(Gs, ranks, title="Ordering", scale=150, shape="dot", node_alpha=0.5):
    pos = dict()
    H = nx.DiGraph()
    all_nodes = set()
    node_colors = {}
    edge_colors = {}
    for i, rank in enumerate(ranks):
        ranks[i] = unit(rank)
    for G in Gs:
        all_nodes |= set(G.nodes())
    for i, rank in enumerate(ranks):
        GG = Gs[i].copy()
        GG.remove_edges_from(list(GG.edges()))
        GG.add_nodes_from(all_nodes)
        new_labels = dict((v, "{}-{}".format(v, i)) for v in all_nodes)
        GG = nx.relabel.relabel_nodes(GG, new_labels)
        pos_i = dict((new_labels[v], (i*scale, r*scale)) for v, r in rank.items())
        pos.update(pos_i)
        H = nx.compose(H, GG)
    for i in range(1, len(ranks)):
        for v, new_rank in ranks[i].items():
            # Calculate change and scale to (-1, 1)
            old_label = "{}-{}".format(v, i-1)
            label = "{}-{}".format(v, i)
            delta = (new_rank - ranks[i-1][v]) / 2.0
            color = gradient(delta)
            node_colors[label] = color
            if old_label in H and label in H:
                H.add_edge(old_label, label)
                edge_colors[(old_label, label)] = color
    return visualize_visjs(
        H, scale=scale, layout=None, pos=pos,
        shape=shape, node_colors=node_colors, node_alpha=node_alpha, shadow=False,
        edge_colors=edge_colors,
        title=title)


In [ ]:
score_t = []
order_t = []
G = []
all_nodes = set()
for wave in range(3):
    G.append(load_tfls(wave+1))
    all_nodes |= set(G[wave].nodes())
for wave in range(3):
    G[wave].add_nodes_from(all_nodes)
    r, o = get_rank(G[wave])
    score_t.append(r)
    order_t.append(o)
# Plot the orderings
plot_ordering_evolution(G, score_t, title="Rank Over Time", scale=1000)

### Behavioral data

In [ ]:
# Helper functions

def load_tfls_behavior():
    # Load alcohol data
    alc = pd.read_csv("external/TFLS/alcohol.csv").set_index("Unnamed: 0")
    alc = alc.rename(columns={"t1": "alcohol1", "t2": "alcohol2", "t3": "alcohol3"})
    df = alc
    # Load age data
    x = pd.read_csv("external/TFLS/age.csv").set_index("Unnamed: 0")
    x = x.rename(columns={"x": "age"})
    df = pd.concat([df, x], axis=1)
    # Load sex data
    x = pd.read_csv("external/TFLS/sex.csv").set_index("Unnamed: 0")
    x = x.rename(columns={"x": "sex"})
    df = pd.concat([df, x], axis=1)
    # Load cannabis data
    x = pd.read_csv("external/TFLS/cannabis.csv").set_index("Unnamed: 0")
    x = x.rename(columns={"t1": "cannabis1", "t2": "cannabis2", "t3": "cannabis3"})
    df = pd.concat([df, x], axis=1)
    # Load money data
    x = pd.read_csv("external/TFLS/money.csv").set_index("Unnamed: 0")
    x = x.rename(columns={"t1": "money1", "t2": "money2", "t3": "money3"})
    df = pd.concat([df, x], axis=1)
    # Load romantic data
    x = pd.read_csv("external/TFLS/romantic.csv").set_index("Unnamed: 0")
    x = x.rename(columns={"t1": "romantic1", "t2": "romantic2", "t3": "romantic3"})
    df = pd.concat([df, x], axis=1)
    # Load tobacco data
    x = pd.read_csv("external/TFLS/tobacco.csv").set_index("Unnamed: 0")
    x = x.rename(columns={"t1": "tobacco1", "t2": "tobacco2", "t3": "tobacco3"})
    df = pd.concat([df, x], axis=1)
    # Set name of index
    df["participant"] = df.index
    df = df.set_index("participant")
    return df

In [ ]:
behavior = load_tfls_behavior()
df = pd.concat([
    behavior["age"],
    behavior["sex"],
    behavior["tobacco1"],
    behavior["cannabis1"],
    behavior["alcohol1"],
    behavior["money1"],
    behavior["romantic1"]
    ], axis=1
)
sr = pd.DataFrame.from_dict(sr_rank, orient="index")
sr["participant"] = sr.index
sr = sr.set_index("participant")
sr = sr.rename(columns={0: "spring_rank"})
df = pd.concat([df, sr], axis=1)
pd.plotting.scatter_matrix(df, figsize=(12,12))
plt.tight_layout()

## References
[BLN2013] De Bacco, C., Larremore, D. B., & Moore, C. (2017). A physical model for efficient ranking in networks. arXiv preprint arXiv:1709.09002.

[MA1997] L. Michell, and A. Amos, "Girls, pecking order and smoking." Social Science & Medicine 44(12), 1861-1869 (1997)

[KONECT2017] Facebook wall posts network dataset -- KONECT, April 2017.

[VMCG2009] Bimal Viswanath, Alan Mislove, Meeyoung Cha, and Krishna P. Gummadi. On the evolution of user interaction in Facebook. In Proc. Workshop on Online Social Networks, pages 37--42, 2009.